In [2]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_openai import  ChatOpenAI
from langchain.tools import tool
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
@tool
def search_temp(query:str):
    """Search for Weather Online."""
    return f"{query}f"

In [4]:
search_temp.name

'search_temp'

In [5]:
class SearchInput(BaseModel):
    query: str = Field(description="Things to Search for")

In [6]:
@tool(args_schema=SearchInput)
def search_temp(query:str):
    """Search for Weather Online."""
    return f"{query}f"

search_temp.args

{'query': {'description': 'Things to Search for',
  'title': 'Query',
  'type': 'string'}}

In [7]:
search_temp.run("KHI?")

'KHI?f'

In [5]:
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location")
    longitude: float = Field(description="Longitude of the location")

In [18]:
import requests
import datetime

@tool(args_schema=OpenMeteoInput)
def get_current_weather(latitude: float, longitude: float):
    """Get the current temperature for a given Coordinates."""
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API REQUEST FAILED: {response.status_code}, {response.text}")
    
    current_utc = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(t.replace("Z", "+00:00")) for t in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc))
    temperature =  temperature_list[closest_time_index]

    return f"The current temperature at {latitude}, {longitude} is {temperature}°C"


In [7]:
get_current_weather.args

{'latitude': {'description': 'Latitude of the location',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location',
  'title': 'Longitude',
  'type': 'number'}}

In [8]:
from langchain.tools.render import format_tool_to_openai_function

search_weather_tool = format_tool_to_openai_function(get_current_weather)

search_weather_tool

C:\Users\Dell\AppData\Local\Temp\ipykernel_8236\899668067.py:3: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  search_weather_tool = format_tool_to_openai_function(get_current_weather)


{'name': 'get_current_weather',
 'description': 'Get the current temperature for a given Coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [12]:
get_current_weather({"latitude": 13, "longitude": 14})

C:\Users\Dell\AppData\Local\Temp\ipykernel_12160\633142176.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_current_weather({"latitude": 13, "longitude": 14})
C:\Users\Dell\AppData\Local\Temp\ipykernel_12160\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


'The current temperature at 13.0, 14.0 is 35.2°C'

In [9]:
import wikipedia

@tool
def search_wikipedia(query:str):
    """Run Wikipedia search & get page summaries."""
    page_titles = wikipedia.search(query)
    summary = []
    for title in page_titles[:3]:
        try: 
            wikipedia_page = wikipedia.page(title=title, auto_suggest=False)
            summary.append(f"Page: {title}, Summary: {wikipedia_page.summary}")
        except Exception as e:
            raise e
    
    if not summary:
        return "No good Wikipedia search results found"
    
    return "\n\n".join(summary)

In [10]:
search_wikipedia.run("What is Langchain ?")

"Page: Retrieval-augmented generation, Summary: Retrieval augmented generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.\n\n\n\nPage: Prompt injection, Summary: Prompt injection is a family of related computer security exploits carried out by getting a machine learning model (such as an LLM) which was trained to follow human-given instructions to follow instructions provided by a malicious user. This stands in contrast to the intended operation of instruction-following systems, 

In [11]:
wikipedia_search_tool = format_tool_to_openai_function(search_wikipedia)

wikipedia_search_tool

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search & get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [12]:
functions = [search_weather_tool, wikipedia_search_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful but sassy assistant."),
    ("user", "{input}"),
])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

model = model.bind(functions=functions)

In [13]:
chain = prompt | model

chain.invoke({"input" : "What is the weather in KHI?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":24.8607,"longitude":67.0011}', 'name': 'get_current_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 102, 'total_tokens': 127, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-c17342d4-72e3-4dc9-8c32-14c2ab136cf6-0', usage_metadata={'input_tokens': 102, 'output_tokens': 25, 'total_tokens': 127, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [14]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

result = chain.invoke({"input" : "What is Langchain ?"})

In [16]:
result.tool


'search_wikipedia'

In [15]:
from langchain.schema.agent import AgentFinish

def routes(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        tools = {
            "get_current_weather": get_current_weather,
            "search_wikipedia": search_wikipedia
        }
        return tools[result.tool].run(result.tool_input)

chain = prompt | model | OpenAIFunctionsAgentOutputParser() | routes

In [16]:
result = chain.invoke({"input" : "What is Langchain ?"})

result

"Page: LangChain, Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax, Summary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Retrieval-augmented generation, Summary: Retrieval augmented generation (RAG) is a technique that grants generative artificial intelligence models information retriev

In [17]:
result = chain.invoke({"input" : "What is the Weather in KHI ?"})

result

C:\Users\Dell\AppData\Local\Temp\ipykernel_8236\3793037649.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc = datetime.datetime.utcnow()


'The current temperature at 24.8607, 67.0011 is 30.0°C'